# Multi-table Datasets - ENRON Archive

## 1. Data import

Connect to the file 'assets/datasets/enron.db' using one of these methods:

- sqlite3 python package
- pandas.read_sql
- SQLite Manager Firefox extension

Take a look at the database and query the master table. How many Tables are there in the db?

> Answer:
There are 3 tables:
- MessageBase
- RecipientBase
- EmployeeBase

In [10]:
import pandas as pd
import sqlite3 as sql
conn = sql.connect("../../assets/datasets/enron.db")
c = conn.cursor()
c.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()

[(u'MessageBase',), (u'RecipientBase',), (u'EmployeeBase',)]

In [11]:
#import pandas as pd
#import sqlite3 as sql

#con = sql.connect("../../assets/datasets/enron.db")

#employeeBase = pd.read_sql_query("SELECT * from EmployeeBase", con)
#messageBase = pd.read_sql_query("SELECT * from MessageBase", con)
#recipientBase = pd.read_sql_query("SELECT * from RecipientBase", con)
# verify that result of SQL query is stored in the dataframe
#employeeBase.head()

Query the `sqlite_master` table to retrieve the schema of the `EmployeeBase` table.

1. What fields are there?
1. What's the type of each of them?

In [12]:
fields = c.execute("SELECT sql FROM sqlite_master WHERE type='table' and name = 'EmployeeBase';").fetchall()
print ''.join(fields[0])

fields = c.execute("SELECT sql FROM sqlite_master WHERE type='table' and name = 'RecipientBase';").fetchall()
print ''.join(fields[0])

fields = c.execute("SELECT sql FROM sqlite_master WHERE type='table' and name = 'MessageBase';").fetchall()
print ''.join(fields[0])

CREATE TABLE EmployeeBase (
                  [eid] INTEGER,
  [name] TEXT,
  [department] TEXT,
  [longdepartment] TEXT,
  [title] TEXT,
  [gender] TEXT,
  [seniority] TEXT
                  
                  )
CREATE TABLE RecipientBase (
    mid INTEGER,
    rno INTEGER,
    to_eid INTEGER,
    
    PRIMARY KEY(mid ASC, rno ASC)
    FOREIGN KEY(mid) REFERENCES Message(mid)
    FOREIGN KEY(to_eid) REFERENCES Employee(eid)
)
CREATE TABLE MessageBase (
    mid INTEGER,
    filename TEXT,
    unix_time INTEGER,
    subject TEXT,
    from_eid INTEGER,
    
    PRIMARY KEY(mid ASC),
    FOREIGN KEY(from_eid) REFERENCES Employee(eid)
)


1. Print the first 5 rows of EmployeeBase table
1. Print the first 5 rows of MessageBase table
1. Print the first 5 rows of RecipientBase table

**Hint**  use `SELECT` and `LIMIT`.

In [13]:
print 'EmployeeBase'
top5EB=c.execute("SELECT * FROM 'EmployeeBase' LIMIT 5;").fetchall()
for row in top5EB:
    print row
print ''
print 'MessageBase'
top5MB=c.execute("SELECT * FROM 'MessageBase' LIMIT 5;").fetchall()
for row in top5MB:
    print row
print ''
print 'RecipientBase'
top5RB=c.execute("SELECT * FROM 'RecipientBase' LIMIT 5;").fetchall()
for row in top5RB:
    print row

EmployeeBase
(1, u'John Arnold', u'Forestry', u'ENA Gas Financial', u'VP Trading', u'Male', u'Senior')
(2, u'Harry Arora', u'Forestry', u'ENA East Power', u'VP Trading', u'Male', u'Senior')
(3, u'Robert Badeer', u'Forestry', u'ENA West Power', u'Mgr Trading', u'Male', u'Junior')
(4, u'Susan Bailey', u'Legal', u'ENA Legal', u'Specialist Legal', u'Female', u'Junior')
(5, u'Eric Bass', u'Forestry', u'ENA Gas Texas', u'Trader', u'Male', u'Junior')

MessageBase
(1, u'taylor-m/sent/11', 910930020, u'Cd$ CME letter', 138)
(2, u'taylor-m/sent/17', 911459940, u'Indemnification', 138)
(3, u'taylor-m/sent/18', 911463840, u'Re: Indemnification', 138)
(4, u'taylor-m/sent/23', 911874180, u'Re: Coral Energy, L.P.', 138)
(5, u'taylor-m/sent/27', 912396120, u'Bankruptcy Code revisions', 138)

RecipientBase
(1, 1, 59)
(2, 1, 15)
(3, 1, 15)
(4, 1, 109)
(4, 2, 49)


Import each of the 3 tables to a Pandas Dataframes

In [14]:
employeeBase = pd.read_sql_query("SELECT * from EmployeeBase", conn)
messageBase = pd.read_sql_query("SELECT * from MessageBase", conn)
recipientBase = pd.read_sql_query("SELECT * from RecipientBase", conn)

In [15]:
employeeBase.head()

,eid,name,department,longdepartment,title,gender,seniority
0,1,John Arnold,Forestry,ENA Gas Financial,VP Trading,Male,Senior
1,2,Harry Arora,Forestry,ENA East Power,VP Trading,Male,Senior
2,3,Robert Badeer,Forestry,ENA West Power,Mgr Trading,Male,Junior
3,4,Susan Bailey,Legal,ENA Legal,Specialist Legal,Female,Junior
4,5,Eric Bass,Forestry,ENA Gas Texas,Trader,Male,Junior


In [16]:
messageBase.head()

,mid,filename,unix_time,subject,from_eid
0,1,taylor-m/sent/11,910930020,Cd$ CME letter,138
1,2,taylor-m/sent/17,911459940,Indemnification,138
2,3,taylor-m/sent/18,911463840,Re: Indemnification,138
3,4,taylor-m/sent/23,911874180,"Re: Coral Energy, L.P.",138
4,5,taylor-m/sent/27,912396120,Bankruptcy Code revisions,138


In [17]:
recipientBase.head()

,mid,rno,to_eid
0,1,1,59
1,2,1,15
2,3,1,15
3,4,1,109
4,4,2,49


## 2. Data Exploration

Use the 3 dataframes to answer the following questions:

1. How many employees are there in the company? **156 employees**
- How many messages are there in the database? **21,635 messages**
- Convert the timestamp column in the messages. When was the oldest message sent? And the newest? **The oldest was sent November 13th, 1998 at 4:07am. The most recent message was sent June 21st, 2002 at 1:37:34pm**
- Some messages are sent to more than one recipient. Group the messages by message_id and count the number of recepients. Then look at the distribution of recepient numbers.
    - How many messages have only one recepient?
    - How many messages have >= 5 recepients?
    - What's the highest number of recepients?
    - Who sent the message with the highest number of recepients?
- Plot the distribution of recepient numbers using Bokeh.

In [18]:
len(employeeBase)

156

In [22]:
len(messageBase)

21635

In [19]:
messageBase['unix_time'] = pd.to_datetime(messageBase['unix_time'],unit='s')

In [20]:
messageBase.sort_values(by='unix_time', ascending=False)

,mid,filename,unix_time,subject,from_eid
21634,21635,germany-c/inbox/23,2002-06-21 13:37:34,Master Termination Log,92
21633,21634,germany-c/inbox/42,2002-06-13 12:16:11,Netting Agreement Materials,4
21632,21633,germany-c/personal/1,2002-06-12 09:21:36,RE: Job Ops,57
21631,21632,germany-c/sent_items/74,2002-06-12 09:20:33,Job Ops,39
21630,21631,germany-c/inbox/49,2002-06-10 11:28:51,NNG Deals,39
21629,21630,germany-c/calendar/5,2002-05-28 15:02:28,Updated: Meeting regarding transport and storage,77
21628,21629,germany-c/inbox/66,2002-05-27 20:32:34,Re: Update,94
21627,21628,germany-c/personal/3,2002-05-22 10:43:37,Sorry for the blank email...here's the real deal,118
21626,21627,germany-c/bankrupt/transport/national_fuel/4,2002-05-20 08:09:45,RE: Nation Fuel Gas Supply Corporation --- E...,77
21625,21626,germany-c/bankrupt/12,2002-05-10 11:40:40,Information Tools Memo,4


In [25]:
#datetimes = messageBase['unix_time'].apply(pd.datetime.fromtimestamp)
#print "first msg was sent on:", min(datetimes)
#print "last msg was sent on:", max(datetimes)

first msg was sent on: _
last msg was sent on: x


In [21]:
messageBase.head()

,mid,filename,unix_time,subject,from_eid
0,1,taylor-m/sent/11,1998-11-13 04:07:00,Cd$ CME letter,138
1,2,taylor-m/sent/17,1998-11-19 07:19:00,Indemnification,138
2,3,taylor-m/sent/18,1998-11-19 08:24:00,Re: Indemnification,138
3,4,taylor-m/sent/23,1998-11-24 02:23:00,"Re: Coral Energy, L.P.",138
4,5,taylor-m/sent/27,1998-11-30 03:22:00,Bankruptcy Code revisions,138


Rescale to investigate the tail of the curve

In [26]:
from collections import Counter

In [28]:
#counts = recipients.groupby('mid')['to_eid'].count().value_counts()
counts = Counter(recipientBase.groupby('mid')['to_eid'].count())
counts

Counter({1: 14985,
         2: 2962,
         3: 1435,
         4: 873,
         5: 711,
         6: 180,
         7: 176,
         8: 61,
         9: 24,
         10: 29,
         11: 47,
         12: 33,
         13: 57,
         14: 11,
         15: 28,
         16: 9,
         17: 2,
         18: 1,
         19: 1,
         20: 1,
         21: 2,
         22: 1,
         24: 1,
         49: 1,
         52: 1,
         55: 1,
         57: 2})

In [29]:
from bokeh.plotting import figure,show,output_notebook
output_notebook()

Loading BokehJS ...

In [30]:
x = [i[0] for i in counts.most_common()]
y = [i[1] for i in counts.most_common()]
left_border = [val-0.5 for val in x]
right_border = [val+0.5 for val in x]


p= figure(title="Message Recipients",tools='',x_axis_label='# of recipients',y_axis_label='Counts')
p.quad(top=y,left=left_border,right=right_border,bottom=0,line_color='black')
show(p)

In [31]:
x = [i[0] for i in counts.most_common()[5:]]  # chop off the first 5
y = [i[1] for i in counts.most_common()[5:]]  # chop off the first 5
left_border = [val-0.5 for val in x]
right_border = [val+0.5 for val in x]


p= figure(title="Message Recipients",tools='',x_axis_label='# of recipients',y_axis_label='Counts')
p.quad(top=y,left=left_border,right=right_border,bottom=0,line_color='black')
show(p)

## 3. Data Merging

Use the pandas merge function to combine the information in the 3 dataframes to answer the following questions:

1. Are there more Men or Women employees?
- How is gender distributed across departments?
- Who is sending more emails? Men or Women?
- What's the average number of emails sent by each gender?
- Are there more Juniors or Seniors?
- Who is sending more emails? Juniors or Seniors?
- Which department is sending more emails? How does that relate with the number of employees in the department?
- Who are the top 3 senders of emails? (people who sent out the most emails)

Answer the following questions regarding received messages:

- Who is receiving more emails? Men or Women?
- Who is receiving more emails? Juniors or Seniors?
- Which department is receiving more emails? How does that relate with the number of employees in the department?
- Who are the top 5 receivers of emails? (people who received the most emails)

In [32]:
employeeBase.gender.value_counts()

Male      113
Female     43
Name: gender, dtype: int64

Which employees sent the most 'mass' emails?

In [34]:
employeeBase.gender.value_counts() / employeeBase.gender.count()

Male      0.724359
Female    0.275641
Name: gender, dtype: float64

In [36]:
# How is gender distributed across departments?
employeeBase.groupby('department')['gender'].value_counts() / employeeBase.groupby('department')['gender'].count()

department  gender
Forestry    Male      0.833333
            Female    0.166667
Legal       Female    0.520000
            Male      0.480000
Other       Male      0.718310
            Female    0.281690
dtype: float64

In [40]:
# Who is sending more emails? Men or Women?
df = pd.merge(employeeBase, messageBase, left_on='eid', right_on='from_eid')
df.gender.value_counts() / df.gender.count()

Male      0.593529
Female    0.406471
Name: gender, dtype: float64

In [41]:
# What's the average number of emails sent by each gender?
df.gender.value_counts() / employeeBase.gender.value_counts()

Male      113.637168
Female    204.511628
Name: gender, dtype: float64

In [43]:
employeeBase.seniority.value_counts()

Junior    82
Senior    74
Name: seniority, dtype: int64

In [44]:
df.seniority.value_counts()

Senior    12439
Junior     9196
Name: seniority, dtype: int64

In [46]:
df.seniority.value_counts() / employeeBase.seniority.value_counts()

Junior    112.146341
Senior    168.094595
Name: seniority, dtype: float64

In [47]:
# Which department is sending more emails? How does that relate with the number of employees in the department?
df.department.value_counts()

Legal       10396
Other        6852
Forestry     4387
Name: department, dtype: int64

In [49]:
df.department.value_counts() / employeeBase.department.value_counts()

Forestry     73.116667
Legal       415.840000
Other        96.507042
Name: department, dtype: float64

In [50]:
# Who are the top 5 senders of emails? (people who sent out the most emails)
top5senders = df.eid.value_counts().head().reset_index()
top5senders.columns = ['eid', 'msgs_sent']
top5senders

,eid,msgs_sent
0,20,1597
1,59,1379
2,120,1142
3,131,859
4,138,658


In [52]:
pd.merge(employeeBase, top5senders, on='eid')

,eid,name,department,longdepartment,title,gender,seniority,msgs_sent
0,20,Jeff Dasovich,Legal,Regulatory and Government Affairs,Director,Male,Senior,1597
1,59,Tana Jones,Legal,ENA Legal,Specialist Legal,Female,Junior,1379
2,120,Sara Shackleton,Legal,ENA Legal,Gen Cnsl Asst,Female,Junior,1142
3,131,James D. Steffes,Legal,Regulatory and Government Affairs,VP of Government Affairs,Male,Senior,859
4,138,Mark E. Taylor,Legal,ENA Legal,VP & Gen Cnsl,Male,Senior,658


Keep exploring the dataset, which other questions would you ask?

Work in pairs. Give each other a challenge and try to solve it.